In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
import dakpy
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

/Users/gem/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.set_printoptions(threshold=np.inf)

In [3]:
#preping the file paths so that each is in numerical order

#change this to where the top level data file is saved
path = '/Users/gem/Desktop/Corrosion Data cropped/'

a = os.listdir(path + 'Mono/Ground Truth')
b = os.listdir(path + 'Mono/Depth')
c = os.listdir(path + 'Non Mono/Ground Truth')
d = os.listdir(path + 'Non Mono/Depth')

a.sort()
b.sort()
c.sort()
d.sort()

a = sorted(a, key=len)
b = sorted(b, key=len)
c = sorted(c, key=len)
d = sorted(d, key=len)

In [8]:
#evaluate the paths
print(a)
print(b)
print(c)
print(d)

['/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_1.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_2.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_3.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_4.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_5.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_6.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_7.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_8.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_9.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_10.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_11.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_12.png', '/Users/gem/Desktop/Corrosion Data cropped/Mono/Ground Truth/gt_mono_13.

In [5]:
#run this cell to remove unwanted items in each list

a.remove('.DS_Store')
b.remove('.DS_Store')
c.remove('.DS_Store')
d.remove('.DS_Store')

In [7]:
#run this cell to append the full path to each item in each list


for i in range(len(a)):
    a[i] = path + 'Mono/Ground Truth/' + a[i]
for i in range(len(b)):
    b[i] = path + 'Mono/Depth/' + b[i]
for i in range(len(c)):
    c[i] = path + 'Non Mono/Ground Truth/' + c[i]
for i in range(len(d)):
    d[i] = path + 'Non Mono/Depth/' + d[i]

In [11]:
#using the dakpy module to get the training images for the mono dataset. Each training image has a raduis of 5
#and a center pixel to classify and is therefore 11X11. That paramter could be changed here in the get_train fuction 
#if a differnt size training patch is desired
data = []
for i, j in zip(a,b):
    data.append(dakpy.get_train(i, j, 5))

In [12]:
#using the dakpy module to get the training images for the non mono dataset.
data1 = []
for i, j in zip(c,d):
    data1.append(dakpy.get_train(i, j, 5))

In [13]:
t = []
for i in range(25):      
    t.append(data[i][1])

In [14]:
t1 = []
for i in range(25):
    t1.append(data1[i][1])

In [15]:
z1_train = np.concatenate(t, axis=0)

In [16]:
z2_train = np.concatenate(t1, axis=0)

In [17]:
h = []
for i in range(25): 
    h.append(data[i][0][:, 0])

In [18]:
h1 = []
for i in range(25):
    h1.append(data1[i][0][:, 0])

In [19]:
w1_train = np.concatenate(h, axis=None)

In [20]:
w2_train = np.concatenate(h1, axis=None)

In [21]:
z_train = np.concatenate((z1_train, z2_train), axis=0)

In [22]:
w_train = np.concatenate((w1_train, w2_train), axis=None)

In [23]:
z_train.shape

(199885, 11, 11)

In [24]:
w_train.shape

(199885,)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(z_train, w_train, test_size = .2)

In [26]:
#Give the negative labels 0 rather than -1

for i in range(len(y_train)):
    if y_train[i] < 0:
        y_train[i] = 0
for i in range(len(y_test)):
    if y_test[i] < 0:
        y_test[i] = 0

In [ ]:
#The CNN


batch_size = 128
num_classes = 2
epochs = 10

# input image dimensions
img_rows, img_cols = 11, 11

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

#parse the data type to 32 bit floats and normalize. The input images have been read in as 16 bit integers from the
#dakpy module
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 65535
x_test /= 65535

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices for use in categorical crossentropy
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='tanh'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (159908, 11, 11, 1)
159908 train samples
39977 test samples
Train on 159908 samples, validate on 39977 samples
Epoch 1/10
159908/159908 [==============================] - 41s 253us/step - loss: 0.2513 - acc: 0.5276 - val_loss: 0.2467 - val_acc: 0.5178
Epoch 2/10
159908/159908 [==============================] - 39s 246us/step - loss: 0.2421 - acc: 0.5706 - val_loss: 0.2481 - val_acc: 0.5466
Epoch 3/10
159908/159908 [==============================] - 47s 295us/step - loss: 0.2406 - acc: 0.5763 - val_loss: 0.2391 - val_acc: 0.5789
Epoch 4/10
159908/159908 [==============================] - 44s 277us/step - loss: 0.2401 - acc: 0.5768 - val_loss: 0.2511 - val_acc: 0.5222
Epoch 5/10
159908/159908 [==============================] - 41s 259us/step - loss: 0.2394 - acc: 0.5784 - val_loss: 0.2505 - val_acc: 0.5178
Epoch 6/10
159908/159908 [==============================] - 42s 261us/step - loss: 0.2388 - acc: 0.5791 - val_loss: 0.2421 - val_acc: 0.5842
Epoch 7/10
159908/159908 [==

In [ ]:
p_1 = model.predict(z1_train.reshape(z1_train.shape[0], 11, 11, 1))

In [ ]:
p_1 = np.delete(p_1, 1, 1)

In [ ]:
p_1 = p_1.reshape(400, 400)

In [ ]:
np.unique(keras.utils.to_categorical(w1_train, 2))